In [1]:
### 피처 가정 세우고 다시 처음부터 데이터 셋 구성
### 배럴 정의 필요

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name())
from tqdm import tqdm
import re

In [2]:
# HTS
t18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding='cp949')
t19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding='cp949')
t20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding='cp949')
t21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding='cp949')

# 경기일정
game = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding='cp949')

# 선수
p18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='cp949')
p19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='cp949')
p20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='cp949')
p21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='cp949')

# 타자 기본
h18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding='cp949')
h19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding='cp949')
h20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding='cp949')
h21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding='cp949')

# 팀
team = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding='cp949')

# 서브미션
submission = pd.read_excel('data/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx')

In [3]:
hts = pd.concat([t18, t19, t20, t21], axis=0).reset_index(drop=True)
player = pd.concat([p18, p19, p20, p21], axis=0).reset_index(drop=True)
hit = pd.concat([h18, h19, h20, h21], axis=0).reset_index(drop=True)

In [4]:
hts.columns = ['연도', '경기코드', '타구코드', '선수코드', '팀코드', '이닝', '타구속도', '발사각도', '타격결과', '상대투수투구구속', '해당구장']
player.columns = ['연도', '선수코드', '선수명', '팀코드', '포지션', '나이', '연봉']
hit.columns = ['연도', '선수코드', '경기', '타석', '타수', '타율', '안타', '홈런', '루타', '장타율', '희생플라이', '볼넷', '삼진', '고의4구', '사구', '병살타']

## 기본 전처리

In [5]:
player = player[-player['연봉'].isnull()]
player['money2'] = player['연봉'].str.replace('[가-힣]','').astype('int64')
player.loc[player['연봉'].str.endswith('달러'), 'money2'] = player.loc[player['연봉'].str.endswith('달러'), 'money2'].apply(lambda x : x * 0.12).astype('int64')
player['연봉'] = player['money2']
player.drop('money2', axis=1, inplace=True)
data = {'연도': 2020, '선수코드': 50802, '선수명':'화이트', '팀코드':'SK', '포지션':'내', '나이': 31, '연봉': 16000}
player = player.append(data, ignore_index=True)
hts.shape, player.shape, hit.shape

((120745, 11), (2635, 7), (1098, 16))

In [6]:
hts['상대팀코드'] = hts.경기코드.str[-5:-1]
for i in tqdm(range(hts.shape[0])):
    hts['상대팀코드'].iloc[i] = hts.iloc[i].상대팀코드.replace(hts.iloc[i].팀코드, '')

100%|█████████████████████████████████████████████████████████████████████████| 120745/120745 [14:54<00:00, 134.95it/s]


In [7]:
hts['날짜'] = '20' + hts.타구코드.str.split('_').str[0] + hts.타구코드.str.split('_').str[1]
hts['날짜'] = pd.to_datetime(hts.날짜)
hts['월'] = hts.날짜.dt.month
hts['일'] = hts.날짜.dt.day
hts['시간'] = hts.날짜.dt.hour
hts['요일'] = hts.날짜.dt.dayofweek
hts['날짜'] = hts['날짜'].astype(str)

In [8]:
# 출루율 = (안타 + 볼넷 + 몸에 맞는 볼) / (타수 + 볼넷 + 몸에 맞는 볼 + 희생플라이)
# on base percentage
hit['출루율'] = (hit['안타'] + hit['볼넷'] + hit['사구']) / (hit['타수'] + hit['볼넷'] + hit['사구'] + hit['희생플라이'])
hit['OPS'] = hit['장타율'] + hit['출루율']
team.columns = ['연도', '팀코드', '팀이름']
hit = hit.merge(player, on=['연도', '선수코드'], how='left')
hit = hit.merge(team, on=['연도', '팀코드'], how='left')

## 가정 - 피처 관련
- 개인역량(배럴타구 / 팀상성 / 이전타석결과 / 기간성적)
- 팀역량(상대수비수준)
- 컨디션(이동거리 / 아시안게임(2018), 올림픽(2021) / 경기시간대 / 지명타자출전 / 몇경기 연속 출장했는지)
- 시계열적 요소 ex)시계열 분해 후 트렌드 적용
- 날씨(7/11까지의 데이터로 예측한다면 data leakage 걱정 안해도 되지 않을까)
- 미디어 지수

### 개인역량

#### 배럴타구 : 타구 속도와 발사각도의 조합상 평균적으로 타율 .500, 장타율 1.500이상을 생산하는 타구들로 정의
* 기존 MLB의 배럴 기준으로는 타구속도가 안나옴(MLB와 KBO의 평균 타구 속도 차이)
* 기대 성적으로 접근하는 방법 사용 : 한국에서 안타 확률 50%이상 장타율 1.500 이상을 기록할 수 있는 타구 스피드와 발사각 구간
* 타구 속도 시속 155-160km, 발사각 22.5-35.0도 / 타구 속도 시속 160-165km, 발사각 20.0-37.5도 / 타구 속도 시속 165km 이상, 발사각 17.5-40.0도
* (출처 : http://www.spotvnews.co.kr/?mod=news&act=articleView&idxno=199017)

In [9]:
hts['타구속도'].describe()

count    120745.000000
mean        133.732175
std          21.337877
min           9.880000
25%         121.680000
50%         137.700000
75%         149.480000
max         210.130000
Name: 타구속도, dtype: float64

In [10]:
# BARREL_KBO식
hts['배럴'] = 0

def BARREL_KBO(x):
    for i in range(hts.shape[0]):       
        if (x['타구속도'][i] >= 155) and (x['타구속도'][i] < 160):
            if (x['발사각도'][i] >= 22.5) and (x['발사각도'][i] <= 35.0):
                hts['배럴'][i] = 1
            else:
                hts['배럴'][i] = 0
        elif (x['타구속도'][i] >= 160) and (x['타구속도'][i] < 165):
            if (x['발사각도'][i] >= 20.0) and (x['발사각도'][i] <= 37.5):
                hts['배럴'][i] = 1
            else:
                hts['배럴'][i] = 0
        elif (x['타구속도'][i] >= 165):
            if (x['발사각도'][i] >= 17.5) and (x['발사각도'][i] <= 40):
                hts['배럴'][i] = 1
            else:
                hts['배럴'][i] = 0
                
BARREL_KBO(hts)

In [11]:
hts['배럴'].value_counts() / hts['배럴'].count()   # 약 2%의 타구

0    0.977639
1    0.022361
Name: 배럴, dtype: float64

In [12]:
hts.query('배럴 == 1')['타격결과'].value_counts()
# 타율 : 0.8921
# 장타율 : 3.3246

홈런       2066
플라이       289
2루타       279
3루타        32
1루타        22
희생플라이      11
직선타         1
Name: 타격결과, dtype: int64

In [13]:
### 출루 시 발사각과 타구속도
hts['타구출루'] = hts['타격결과'].apply(lambda x : 1 if x in ['1루타', '2루타', '3루타', '홈런'] else 0)
hts.groupby(['선수코드', '타구출루'])['발사각도'].describe().unstack()

count              mean                   std              min        \
타구출루       0      1          0          1          0          1     0     1   
선수코드                                                                          
50054   40.0   15.0  20.702500   8.373333  25.730193  14.820568 -19.5 -15.2   
50066   11.0    6.0  17.245455  12.366667  27.958697  11.649149 -13.1  -0.6   
50150   19.0   11.0  24.436842  17.790909  28.726279  14.329302 -23.7  -1.6   
50165  249.0  158.0  27.530522  18.539873  26.520499  13.547590 -26.9 -13.5   
50167    5.0    1.0  43.660000  11.700000  22.305224        NaN  25.9  11.7   
...      ...    ...        ...        ...        ...        ...   ...   ...   
79530   21.0   10.0  22.552381   8.870000  31.991790  13.287258 -19.6 -14.1   
79608  748.0  442.0  21.900936  11.045701  29.536770  12.446251 -62.3 -24.2   
79705   84.0   45.0  26.851190  15.755556  29.933700  14.083542 -36.6  -9.0   
99606   68.0   51.0  15.025000  10.643137  28.358654  11.563153 -54.5 -14.0   
99810  138.0   80.0  15.392029   8.561250  29.380330  14.066861 -43.7 -29.6   

          25%            50%            75%           max        
타구출루        0       1      0      1       0       1     0     1  
선수코드                                                             
50054  -0.825   5.600  21.65   7.10  34.350   9.700  83.7  53.4  
50066  -5.900   6.150   8.40   9.80  40.500  15.550  59.5  32.8  
50150   1.250  10.400  23.70  19.70  42.050  21.200  74.3  48.8  
50165   3.800   9.425  27.60  17.45  47.000  27.350  83.6  59.1  
50167  29.000  11.700  31.10  11.70  54.100  11.700  78.2  11.7  
...       ...     ...    ...    ...     ...     ...   ...   ...  
79530  -6.100   8.525   9.80  10.00  48.400  12.225  86.6  36.4  
79608  -3.800   3.700  21.20  10.40  44.125  19.275  87.4  54.1  
79705   0.275   5.400  33.80  14.80  51.975  27.100  85.1  42.6  
99606  -5.975   2.750   6.05  11.20  37.250  18.900  84.5  38.6  
99810  -7.650   1.000  12.40   9.55  36.625  16.300  85.9  45.9  

[437 rows x 16 columns]

#### 팀 상성

In [14]:
hit_log = pd.read_csv('data/hit_log.csv', index_col=0) # 타자 당 날짜별 기록
hit_log['날짜'] = hit_log['연도'].astype(int).astype(str) + '-' + hit_log['ss'].astype(str)
hit_log['상대팀코드'] = hit_log['상대'].apply(lambda x : 'KT' if ('KT' in x)
                                       else 'LG' if ('LG' in x)
                                       else 'NC' if ('NC' in x)
                                       else 'HT' if ('KIA' in x)
                                       else 'SK' if (('SK' in x) or ('SSG' in x))
                                       else 'SS' if ('삼성' in x)
                                       else 'LT' if ('롯데' in x)
                                       else 'OB' if ('두산' in x)
                                       else 'HH' if ('한화' in x)
                                       else 'WO')
hit_log['스코어'] = hit_log['결과'].apply(lambda x : x.split(' ')[1]) 
hit_log['결과'] = hit_log['결과'].apply(lambda x : x.split(' ')[0])

# avLI - 얼마나 중요한 상황에 등판했는가(평균)
# RE24 - 상황대비 기대득점
# WPA - 승리 확률 기여도(두 상황 간의 기대승률 차이)
hit_log['WPA/avLI'] = (hit_log['WPA'] / hit_log['avLl']).fillna(0)

In [15]:
hit_log = hit_log.query('날짜 <= "2021-07-11"')

In [16]:
hts = hts.merge(player.drop('팀코드', axis=1), on=['선수코드', '연도'], how='left')
hts['날짜'] = hts['날짜'].str[:10]
hit_log['날짜'] = pd.to_datetime(hit_log.날짜)
hit_log['월'] = hit_log.날짜.dt.month
hit_log['일'] = hit_log.날짜.dt.day
hit_log['요일'] = hit_log.날짜.dt.dayofweek
hit_log['날짜'] = hit_log['날짜'].astype(str)

In [17]:
hts1 = hts[['상대팀코드', '팀코드', '날짜']].drop_duplicates()
hit_log = hit_log.merge(hts1, on=['날짜', '상대팀코드'], how='left')

player = player.query('포지션 != "투"')
hit_log = hit_log.merge(player, on=['연도', '팀코드', '선수명'], how='left')

In [18]:
hit_log = hit_log.fillna(method='ffill')

In [19]:
hit_log['일타율'] = ((hit_log['안타'] + hit_log['2타'] + hit_log['3타'] + hit_log['홈런']) / hit_log['타수']).fillna(0)

In [20]:
hit_log

,ss,상대,결과,타순,P,선발,타수,득점,안타,2타,...,WPA/avLI,월,일,요일,팀코드,선수코드,포지션,나이,연봉,일타율
0,07-19,@KT,L,8,PH,0,1,0,1,0,...,0.057088,7,19,3,HH,60184.0,포,27.0,2700.0,1.000000
1,07-22,@삼성,L,8,C,1,3,0,0,0,...,-0.075728,7,22,6,HH,60184.0,포,27.0,2700.0,0.000000
2,09-06,@KT,W,5,PR,0,2,1,1,1,...,-0.050000,9,6,3,HH,60184.0,포,27.0,2700.0,1.000000
3,09-07,@KT,L,6,PH,0,1,0,0,0,...,0.000000,9,7,4,HH,60184.0,포,27.0,2700.0,0.000000
4,09-09,@LG,L,8,PH,0,2,0,0,0,...,-0.050980,9,9,6,HH,60184.0,포,27.0,2700.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63743,07-04,한화,W,6,1B,1,3,1,1,0,...,0.062791,7,4,6,LG,69102.0,내,20.0,3000.0,0.333333
63744,07-05,한화,W,2,1B,1,4,1,2,0,...,0.179870,7,5,0,LG,69102.0,내,20.0,3000.0,0.500000
63745,07-02,@KT,L,8,1B,1,3,0,1,0,...,0.014394,7,2,4,WO,69366.0,내,21.0,3000.0,0.333333
63746,07-04,@KT,L,8,1B,1,2,0,0,0,...,-0.047826,7,4,6,WO,69366.0,내,21.0,3000.0,0.000000


In [21]:
hit_log.groupby(['선수코드', '상대팀코드'])['일타율'].mean().unstack().fillna(0)

상대팀코드,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
선수코드,,,,,,,,,,
50054.0,0.111111,0.179487,0.000000,0.250000,0.074074,0.000000,0.212121,0.046875,0.071429,0.075000
50066.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.625000,0.000000,0.187500,0.050000
50150.0,0.214286,0.000000,0.444444,0.000000,0.833333,0.000000,0.100000,0.125000,0.333333,0.000000
50165.0,0.327778,0.426190,0.391111,0.000000,0.287778,0.435417,0.447826,0.480952,0.314035,0.278986
50167.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...
79530.0,0.000000,0.000000,0.555556,0.500000,0.000000,0.166667,0.000000,0.194444,0.000000,1.000000
79608.0,0.400694,0.373333,0.288492,0.388462,0.370513,0.401020,0.407246,0.377244,0.438889,0.479433
79705.0,0.000000,0.288462,0.434524,0.214286,0.192708,0.115385,0.065385,0.255952,0.250000,0.189394


In [22]:
hts.groupby(['선수코드', '상대팀코드'])['배럴'].mean().unstack().fillna(0)

상대팀코드,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
선수코드,,,,,,,,,,
50054,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50066,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50150,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50165,0.085106,0.20000,0.054054,0.000000,0.028571,0.111111,0.148148,0.102041,0.152174,0.111111
50167,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
79530,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
79608,0.014599,0.04878,0.000000,0.000000,0.000000,0.006993,0.006623,0.006410,0.006579,0.016260
79705,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
hts.groupby(['선수코드', '상대팀코드'])['타구출루'].mean().unstack().fillna(0)

상대팀코드,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
선수코드,,,,,,,,,,
50054,0.000000,0.555556,0.000000,0.333333,0.166667,0.000000,0.500000,0.222222,0.166667,0.250000
50066,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.666667,0.333333
50150,0.250000,0.000000,0.500000,0.000000,0.600000,0.000000,0.250000,0.333333,0.500000,0.000000
50165,0.382979,0.500000,0.324324,0.000000,0.228571,0.466667,0.462963,0.469388,0.304348,0.314815
50167,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
79530,0.000000,0.000000,0.600000,1.000000,0.000000,0.333333,0.000000,0.428571,0.000000,1.000000
79608,0.364964,0.341463,0.330357,0.340000,0.373333,0.384615,0.364238,0.333333,0.407895,0.447154
79705,0.000000,0.428571,0.625000,0.214286,0.375000,0.200000,0.272727,0.263158,0.181818,0.444444


In [24]:
hit_log.groupby(['선수코드', '상대팀코드'])['WPA/avLI'].mean().unstack().fillna(0)

상대팀코드,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
선수코드,,,,,,,,,,
50054.0,-0.005858,0.015270,0.000000,0.029362,-0.022173,-0.013480,0.006296,-0.004595,-0.011865,-0.028831
50066.0,-0.027419,-0.016667,0.000000,-0.011216,-0.006452,0.000000,0.008240,0.000000,0.008298,-0.042346
50150.0,-0.009893,0.000000,-0.043556,0.000000,0.029739,-0.014609,0.020857,0.012188,0.010027,0.000000
50165.0,0.044655,0.041631,-0.006187,0.000000,-0.011628,0.033044,0.016853,0.067323,0.013812,-0.006435
50167.0,0.000000,0.000000,0.024868,0.000000,-0.028364,-0.024860,-0.025000,-0.025110,-0.012667,-0.004062
...,...,...,...,...,...,...,...,...,...,...
79530.0,-0.045094,-0.028487,-0.013372,0.039212,-0.032528,-0.074885,-0.025000,-0.017096,-0.057143,0.000000
79608.0,0.017962,-0.021800,0.011277,0.012855,0.039620,0.022885,0.013045,0.005809,0.021705,0.045345
79705.0,0.000000,-0.009525,0.076583,-0.004719,0.006970,0.013759,-0.014715,-0.028852,-0.005231,-0.008923


#### 기간성적

In [25]:
# 요일별 성적 -> 더미 변수로
hit_log.groupby(['선수코드', '요일'])['WPA/avLI'].mean().unstack().fillna(0)

요일,0,1,2,3,4,5,6
선수코드,,,,,,,
50054.0,-0.025000,0.007835,-0.004295,0.002202,-0.006992,-0.002135,-0.026534
50066.0,0.000000,-0.025012,-0.021159,-0.033474,-0.026669,-0.001796,0.045779
50150.0,-0.010000,0.016817,-0.016888,0.006342,0.001363,-0.036062,0.023000
50165.0,-0.040550,0.011950,0.070989,0.046296,0.013588,0.016785,-0.015862
50167.0,0.000000,-0.008125,-0.029071,0.043970,-0.034783,-0.020010,-0.015833
...,...,...,...,...,...,...,...
79530.0,0.000000,0.002065,-0.039888,-0.040094,-0.015679,-0.012637,-0.039312
79608.0,0.208731,0.026351,0.006493,0.008711,0.033627,0.010635,0.021127
79705.0,-0.082778,0.021808,0.009861,-0.012202,-0.006141,0.015317,-0.005670


In [26]:
hit_log.groupby(['선수코드', '요일'])['일타율'].mean().unstack().fillna(0)

요일,0,1,2,3,4,5,6
선수코드,,,,,,,
50054.0,0.000000,0.096154,0.243590,0.093750,0.027778,0.093750,0.108974
50066.0,0.000000,0.166667,0.000000,0.000000,0.050000,0.333333,0.687500
50150.0,0.000000,0.166667,0.625000,0.388889,0.080000,0.000000,0.300000
50165.0,0.208333,0.393333,0.489744,0.409140,0.366667,0.300617,0.336420
50167.0,0.000000,1.000000,0.000000,0.250000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
79530.0,0.000000,0.111111,0.062500,0.333333,0.500000,0.166667,0.000000
79608.0,0.625000,0.474648,0.362381,0.336667,0.423810,0.345833,0.428974
79705.0,0.000000,0.250000,0.250694,0.100000,0.214912,0.458333,0.083333


In [27]:
hit_log.groupby(['선수코드', 'P'])['WPA/avLI'].mean().unstack().fillna(0)

P,1B,2B,3B,C,CF,DH,LF,P,PH,PR,RF,SS
선수코드,,,,,,,,,,,,
50054.0,-0.016667,-0.013139,0.008221,0.000000,0.0,-0.023529,0.0,0.0,-0.024091,0.004700,0.000000,-0.027512
50066.0,0.000000,0.000000,0.000000,-0.007996,0.0,0.000000,0.0,0.0,-0.062676,0.000000,0.000000,0.000000
50150.0,0.035000,-0.025000,-0.002430,0.000000,0.0,0.000000,0.0,0.0,0.040000,0.000498,0.000000,-0.017847
50165.0,0.021908,0.000000,0.000000,0.000000,0.0,0.028870,0.0,0.0,0.000239,0.000000,0.000000,0.000000
50167.0,0.009292,0.000000,0.000000,0.000000,0.0,-0.015833,0.0,0.0,-0.031842,-0.002031,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
79530.0,-0.018475,-0.027850,-0.037593,0.000000,0.0,0.000000,0.0,0.0,-0.029167,0.020000,0.000000,0.000000
79608.0,-0.001315,0.021874,0.000000,0.000000,0.0,0.002045,0.0,0.0,-0.000699,0.000000,0.000000,0.000000
79705.0,-0.009486,0.000000,0.011701,0.000000,0.0,0.000000,0.0,0.0,0.023137,0.060000,0.000000,0.000000


In [28]:
hit_log.groupby(['선수코드', 'P'])['일타율'].mean().unstack().fillna(0)

P,1B,2B,3B,C,CF,DH,LF,P,PH,PR,RF,SS
선수코드,,,,,,,,,,,,
50054.0,0.000000,0.102151,0.149123,0.000000,0.0,0.666667,0.0,0.0,0.031250,0.057692,0.00000,0.125
50066.0,0.000000,0.000000,0.000000,0.229167,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000
50150.0,1.000000,0.000000,0.384848,0.000000,0.0,0.000000,0.0,0.0,0.250000,0.166667,0.00000,0.000
50165.0,0.373543,0.000000,0.000000,0.000000,0.0,0.374667,0.0,0.0,0.666667,0.000000,0.00000,0.000
50167.0,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.00000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
79530.0,0.190476,0.148148,0.135417,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.600000,0.00000,0.000
79608.0,0.229412,0.418021,0.000000,0.000000,0.0,0.313542,0.0,0.0,0.260000,0.000000,0.00000,0.000
79705.0,0.123239,0.000000,0.296259,0.000000,0.0,0.000000,0.0,0.0,0.479167,0.500000,0.00000,0.000


In [29]:
# 최근 3일치 성적
hit_log['전날성적_일타율'] = (hit_log.groupby('선수코드')['일타율'].shift(1))
hit_log['전날성적_기여도'] = hit_log.groupby('선수코드')['WPA/avLI'].shift(1)

hit_log['2전날성적_일타율'] = hit_log.groupby('선수코드')['일타율'].shift(2)
hit_log['2전날성적_기여도'] = hit_log.groupby('선수코드')['WPA/avLI'].shift(2)

hit_log['3전날성적_일타율'] = hit_log.groupby('선수코드')['일타율'].shift(3)
hit_log['3전날성적_기여도'] = hit_log.groupby('선수코드')['WPA/avLI'].shift(3)

In [30]:
# 최근 3경기 성적
hit_log['최근 3경기 성적_일타율'] = (hit_log['전날성적_일타율'] + hit_log['2전날성적_일타율'] + hit_log['3전날성적_일타율']) / 3
hit_log['최근 3경기 성적_기여도'] = (hit_log['전날성적_기여도'] + hit_log['2전날성적_기여도'] + hit_log['3전날성적_기여도']) / 3

In [31]:
hit_log.head(10)

,ss,상대,결과,타순,P,선발,타수,득점,안타,2타,...,연봉,일타율,전날성적_일타율,전날성적_기여도,2전날성적_일타율,2전날성적_기여도,3전날성적_일타율,3전날성적_기여도,최근 3경기 성적_일타율,최근 3경기 성적_기여도
0,07-19,@KT,L,8,PH,0,1,0,1,0,...,2700.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07-22,@삼성,L,8,C,1,3,0,0,0,...,2700.0,0.0,1.0,0.057088,NaN,NaN,NaN,NaN,NaN,NaN
2,09-06,@KT,W,5,PR,0,2,1,1,1,...,2700.0,1.0,0.0,-0.075728,1.0,0.057088,NaN,NaN,NaN,NaN
3,09-07,@KT,L,6,PH,0,1,0,0,0,...,2700.0,0.0,1.0,-0.050000,0.0,-0.075728,1.0,0.057088,0.666667,-0.022880
4,09-09,@LG,L,8,PH,0,2,0,0,0,...,2700.0,0.0,0.0,0.000000,1.0,-0.050000,0.0,-0.075728,0.333333,-0.041909
5,10-04,롯데,L,6,PH,0,1,0,0,0,...,2700.0,0.0,0.0,-0.050980,0.0,0.000000,1.0,-0.050000,0.333333,-0.033660
6,04-21,삼성,L,7,C,0,1,0,0,0,...,3000.0,0.0,0.0,-0.025000,0.0,-0.050980,0.0,0.000000,0.000000,-0.025327
7,04-24,롯데,W,4,PR,0,0,0,0,0,...,3000.0,0.0,0.0,0.000000,0.0,-0.025000,0.0,-0.050980,0.000000,-0.025327
8,04-27,@NC,L,8,C,0,1,0,0,0,...,3000.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-0.025000,0.000000,-0.008333
9,04-27,@LG,L,9,PH,0,1,0,0,0,...,3000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 컨디션

#### 이동거리

In [32]:
hit_log['홈'] = 0

def home(hit_log):
    for i in range(hit_log.shape[0]):
        if '@' in hit_log['상대'][i]:
            hit_log['홈'][i] = hit_log['상대팀코드'][i]
        else:
            hit_log['홈'][i] = hit_log['팀코드'][i]
            
home(hit_log)

In [33]:
hit_log['구장'] = 0

hit_log.loc[hit_log['홈'] == 'NC', '구장'] = '창원'
hit_log.loc[(hit_log['연도'] == 2018) & (hit_log['홈'] == 'NC'), '구장'] = '마산'
hit_log.loc[hit_log['홈'] == 'SK', '구장'] = '문학'
hit_log.loc[hit_log['홈'] == 'SS', '구장'] = '대구'
hit_log.loc[hit_log['홈'] == 'LT', '구장'] = '부산'
hit_log.loc[hit_log['홈'] == 'KT', '구장'] = '수원'
hit_log.loc[hit_log['홈'] == 'HH', '구장'] = '대전'
hit_log.loc[hit_log['홈'] == 'HT', '구장'] = '광주'
hit_log.loc[hit_log['홈'] == 'WO', '구장'] = '고척'
hit_log.loc[hit_log['홈'] == 'OB', '구장'] = '잠실'
hit_log.loc[hit_log['홈'] == 'LG', '구장'] = '잠실'

In [34]:
hit_log['이전구장'] = hit_log.groupby('선수코드')['구장'].shift(1)

In [35]:
# 최근 일주일 이동거리 / 최근 3일간 이동거리
수도권 = ['잠실', '문학', '수원', '고척']
충청도 = ['대전']
전라도 = ['광주']
경상도 = ['대구', '부산', '마산', '창원']

In [36]:
# 수도권 -> 수도권 : 1
# 수도권 -> 충청도 : 2
# 수도권 -> 전라도 : 3
# 수도권 -> 경상도 : 4
# 충청도 -> 전라도 : 2
# 충청도 -> 경상도 : 3
# 전라도 -> 경상도 : 2
# 경상도 -> 경상도 : 1

In [37]:
def distance_weight(hit_log):
    hit_log['이동거리_가중치'] = 0
    for i in range(hit_log.shape[0]):
        if (hit_log['이전구장'][i] in 수도권) and (hit_log['구장'][i] in 수도권):
            hit_log['이동거리_가중치'][i] = 1
            
        elif (hit_log['이전구장'][i] in 수도권) and (hit_log['구장'][i] in 충청도):
            hit_log['이동거리_가중치'][i] = 1   
        elif (hit_log['이전구장'][i] in 충청도) and (hit_log['구장'][i] in 수도권):
            hit_log['이동거리_가중치'][i] = 1
            
        elif (hit_log['이전구장'][i] in 수도권) and (hit_log['구장'][i] in 전라도):
            hit_log['이동거리_가중치'][i] = 3
        elif (hit_log['이전구장'][i] in 전라도) and (hit_log['구장'][i] in 수도권):
            hit_log['이동거리_가중치'][i] = 3
            
        elif (hit_log['이전구장'][i] in 수도권) and (hit_log['구장'][i] in 경상도):
            hit_log['이동거리_가중치'][i] = 4
        elif (hit_log['이전구장'][i] in 경상도) and (hit_log['구장'][i] in 수도권):
            hit_log['이동거리_가중치'][i] = 4
            
        elif (hit_log['이전구장'][i] in 충청도) and (hit_log['구장'][i] in 전라도):
            hit_log['이동거리_가중치'][i] = 2
        elif (hit_log['이전구장'][i] in 전라도) and (hit_log['구장'][i] in 충청도):
            hit_log['이동거리_가중치'][i] = 2
        
        elif (hit_log['이전구장'][i] in 충청도) and (hit_log['구장'][i] in 경상도):
            hit_log['이동거리_가중치'][i] = 3
        elif (hit_log['이전구장'][i] in 경상도) and (hit_log['구장'][i] in 충청도):
            hit_log['이동거리_가중치'][i] = 3
        
        elif (hit_log['이전구장'][i] in 전라도) and (hit_log['구장'][i] in 경상도):
            hit_log['이동거리_가중치'][i] = 2
        elif (hit_log['이전구장'][i] in 경상도) and (hit_log['구장'][i] in 전라도):
            hit_log['이동거리_가중치'][i] = 2
            
        elif (hit_log['이전구장'][i] in 경상도) and (hit_log['구장'][i] in 경상도):
            hit_log['이동거리_가중치'][i] = 1
            
        else:
            hit_log['이동거리_가중치'][i] = 0
            
distance_weight(hit_log)

In [38]:
hit_log.head()

,ss,상대,결과,타순,P,선발,타수,득점,안타,2타,...,2전날성적_일타율,2전날성적_기여도,3전날성적_일타율,3전날성적_기여도,최근 3경기 성적_일타율,최근 3경기 성적_기여도,홈,구장,이전구장,이동거리_가중치
0,07-19,@KT,L,8,PH,0,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,KT,수원,NaN,0
1,07-22,@삼성,L,8,C,1,3,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,SS,대구,수원,4
2,09-06,@KT,W,5,PR,0,2,1,1,1,...,1.0,0.057088,NaN,NaN,NaN,NaN,KT,수원,대구,4
3,09-07,@KT,L,6,PH,0,1,0,0,0,...,0.0,-0.075728,1.0,0.057088,0.666667,-0.022880,KT,수원,수원,1
4,09-09,@LG,L,8,PH,0,2,0,0,0,...,1.0,-0.050000,0.0,-0.075728,0.333333,-0.041909,LG,잠실,수원,1


#### 국가대표 차출 여부(아시안게임, 올림픽)

In [39]:
# 2018 아시안게임
# 2020 올림픽(2021)

2018 아시안게임 명단(24) - 2018-08-18 ~ 09-02
* 투수(11) : 양현종, 임기영(이상 KIA), 박치국, 이용찬, 함덕주(이상 두산), 박종훈(SK), 임찬규(이상 LG), 정우람(한화), 최충연(삼성), 최원태, 장필준
* 포수(2) : 양의지(두산), 이재원(SK)
* 내야수(6) : 안치홍(KIA), 박민우(NC), 오지환(LG), 김하성, 박병호(이상 넥센), 황재균(KT)
* 외야수(5) : 김재환(두산), 손아섭(롯데), 김현수(LG), 박해민(삼성), 이정후(키움)

2020 도쿄올림픽 명단(24) - 2021-07-21 ~ 08-08
* 투수(11) : 최원준(두산), 고영표(kt), 고우석, 차우찬(이상 LG), 조상우(키움), 박세웅, 김진욱(이상 롯데), 오승환, 원태인(이상 삼성), 김민우(한화), 이의리(KIA)
* 포수(2) : 양의지(NC), 강민호(삼성)
* 내야수(7) : 강백호, 황재균(이상 kt), 오재일(삼성), 최주환(SSG), 허경민(두산), 오지환(LG), 김혜성(키움)
* 외야수(4) : 박건우(두산), 김현수(LG), 이정후(키움), 박해민(삼성)

In [40]:
# 대회 참여한 애들 참여 후 성적 확인하기

In [41]:
lst_18 = ['양의지', '이재원', '안치홍', '박민우', '오지환', '김하성', '박병호', '황재균', '김재환', '손아섭', '김현수', '박해민', '이정후']
lst_20 = ['양의지', '강민호', '강백호', '황재균', '오재일', '최주환', '허경민', '오지환', '김혜성', '박건우', '김현수', '이정후', '박해민']

In [42]:
hts['18_아시안게임'] = 0
hts.loc[(hts['선수명'].isin(lst_18)) & (hts['연도']==2018) & (hts['날짜']>="2018-09-02"), '18_아시안게임'] = 1

In [43]:
hts['18_아시안게임'].value_counts()

0    119815
1       930
Name: 18_아시안게임, dtype: int64

In [44]:
hit_log['18_아시안게임'] = 0
hit_log.loc[(hit_log['선수명'].isin(lst_18)) & (hit_log['연도']==2018) & (hit_log['날짜']>="2018-09-02"), '18_아시안게임'] = 1

In [45]:
hit_log['18_아시안게임'].value_counts()

0    63368
1      380
Name: 18_아시안게임, dtype: int64

In [46]:
a = hit_log.query('선수명 in @lst_18 and 연도==2018').groupby(['선수코드', '18_아시안게임'])['일타율'].mean().unstack().fillna(0)

# 76290 김현수는 아시안게임 이후 부상
a = a.query('선수코드 != 76290')
a[1]-a[0]

선수코드
62415.0    0.160735
62907.0    0.118659
64300.0   -0.073480
67341.0   -0.093524
75125.0   -0.042619
76232.0   -0.098700
76313.0    0.083590
76812.0   -0.070578
77532.0   -0.117581
78224.0    0.026697
79109.0    0.028407
79608.0   -0.108978
dtype: float64

In [47]:
np.mean(a[1]-a[0])

-0.015614320726579861

#### 경기 시간대

In [48]:
hts.head()

,연도,경기코드,타구코드,선수코드,팀코드,이닝,타구속도,발사각도,타격결과,상대투수투구구속,...,일,시간,요일,배럴,타구출루,선수명,포지션,나이,연봉,18_아시안게임
0,2018,20180324HHWO0,180324_140436,62797,HH,1,131.50,42.7,플라이,149.59,...,24,14,5,0,0,양성우,외,29,10000,0
1,2018,20180324HHWO0,180324_140514,76753,HH,1,135.18,9.9,1루타,148.78,...,24,14,5,0,1,송광민,내,35,24000,0
2,2018,20180324HHWO0,180324_140647,71752,HH,1,152.41,2.1,1루타,148.59,...,24,14,5,0,1,김태균,내,36,140000,0
3,2018,20180324HHWO0,180324_140911,62700,HH,1,113.72,13.5,1루타,139.13,...,24,14,5,0,1,하주석,내,24,12000,0
4,2018,20180324HHWO0,180324_142050,68730,HH,2,54.11,16.4,번트안타,134.07,...,24,14,5,0,0,호잉,외,29,48000,0


In [49]:
hts.groupby(['선수코드', '시간'])['배럴'].mean().unstack().fillna(0)

시간,13,14,15,16,17,18,19,20,21,22,23
선수코드,,,,,,,,,,,
50054,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
50066,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
50150,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
50165,0.0,0.173913,0.050000,0.1,0.111111,0.106667,0.126316,0.100000,0.113208,0.25,0.0
50167,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
79530,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
79608,0.0,0.000000,0.000000,0.0,0.012500,0.011905,0.010101,0.008163,0.006250,0.00,0.0
79705,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0


In [50]:
hts.groupby(['선수코드', '시간'])['타구출루'].mean().unstack().fillna(0)

시간,13,14,15,16,17,18,19,20,21,22,23
선수코드,,,,,,,,,,,
50054,0.0,1.000000,0.250000,0.000000,0.000000,0.000000,0.500000,0.363636,0.125000,0.000000,0.0
50066,0.0,0.000000,1.000000,0.333333,1.000000,1.000000,0.000000,0.250000,0.200000,0.000000,0.0
50150,0.0,0.000000,1.000000,0.000000,0.500000,0.000000,0.400000,0.444444,0.250000,0.000000,0.0
50165,0.0,0.304348,0.150000,0.550000,0.407407,0.306667,0.463158,0.422222,0.358491,0.500000,0.0
50167,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
79530,0.0,0.000000,0.000000,0.000000,0.000000,0.285714,0.333333,0.125000,1.000000,0.000000,0.0
79608,0.0,0.395349,0.340000,0.414634,0.400000,0.325397,0.360269,0.346939,0.456250,0.500000,0.8
79705,0.0,0.375000,0.500000,0.250000,0.166667,0.437500,0.280000,0.400000,0.363636,0.166667,0.0


### 시계열 분해

In [51]:
import statsmodels.api as sm

In [ ]:
hit_log['날짜'] = pd.to_datetime(hit_log['날짜'])
hit_log.set_index('날짜', inplace=True)

a = hit_log.query('선수명 == "김현수" and 연도 == 2018')
a = a.loc[a.index.duplicated() == False]
a = a.asfreq('D', method='ffill')

plt.rcParams['figure.figsize'] = (14, 9)
sm.tsa.seasonal_decompose(a['일타율'], model='additive').plot()
plt.show()

In [62]:
result = sm.tsa.seasonal_decompose(a['일타율'], model='additive')
Y_trend = pd.DataFrame(result.trend)
Y_trend.fillna(method='ffill', inplace=True)
Y_trend.fillna(method='bfill', inplace=True)
Y_trend.columns = ['일타율_trend']
Y_trend['날짜'] = Y_trend.index
Y_trend = Y_trend.reset_index(drop=True)

In [63]:
Y_trend

,일타율_trend,날짜
0,0.321429,2018-03-24
1,0.321429,2018-03-25
2,0.321429,2018-03-26
3,0.321429,2018-03-27
4,0.357143,2018-03-28
...,...,...
160,1.000000,2018-08-31
161,0.857143,2018-09-01
162,0.857143,2018-09-02
163,0.857143,2018-09-03


In [52]:
def trend_decompose(hit_log, column):
    hit_log1 = hit_log.copy()
    hit_log1['날짜'] = pd.to_datetime(hit_log1['날짜'])
    hit_log1.set_index('날짜', inplace=True)
    
    for i in list(hit_log['선수코드'].unique()):
        
        a = hit_log1.loc[hit_log1['선수코드'] == i]
        a = a.loc[a.index.duplicated() == False]
        a = a.asfreq('D', method='ffill')
        
        result = sm.tsa.seasonal_decompose(a[column], model='additive', period=int(len(a)/2))
        Y_trend = pd.DataFrame(result.trend)
        Y_trend.fillna(method='ffill', inplace=True)
        Y_trend.fillna(method='bfill', inplace=True)
        Y_trend.columns = [column + '_trend']
        Y_trend['선수코드'] = i
        Y_trend['날짜'] = Y_trend.index
        Y_trend = Y_trend.reset_index(drop=True)
        
        hit_log['날짜'] = pd.to_datetime(hit_log['날짜'])
        
        hit_log = hit_log.merge(Y_trend, on=['날짜', '선수코드'], how='left')
    
    return hit_log

In [53]:
hit_log = trend_decompose(hit_log, 'WPA/avLI')

ValueError: index must be monotonic increasing or decreasing

In [ ]:
hit_log.columns